## Using Amazon Forecast to predict P&G future sales in Oman
Chirayu Khimji

MSc Applied Computational Science and Engineering

Imperial College London


Note: This notebook will not run on other machines due to privacy aspect of uploading P&G data (data is stored locally on my machine), its purpose is just to showcase the power of data science for KR CPG group.

The overall process for using Amazon Forecast is the following:

1. Create a Dataset Group, this is the large box that isolates models and the data they are trained on from each other.
2. Create a Dataset, in Forecast there are 3 types of dataset, Target Time Series, Related Time Series, and Item Metadata. The Target Time Series is required, the others provide additional context with certain algorithms. 
3. Import data, this moves the information from S3 into a storage volume where the data can be used for training and validation.
4. Train a model, Forecast automates this process for you but you can also select particular algorithms, and you can provide your own hyper parameters or use Hyper Parameter Optimization(HPO) to determine the most performant values for you.
5. Deploy a Predictor, here you are deploying your model so you can use it to generate a forecast.
6. Query the Forecast, given a request bounded by time for an item, return the forecast for it. Once you have this you can evaluate its performance or use it to guide your decisions about the future.

In this notebook we walk through the steps outlined above. One additional task that will be done here is to trim part of our training and validation data so that we can measure the accuracy of a forecast against our predictions. 


## Table Of Contents
* Setup
* Data Preparation
* Creating the Dataset Group and Dataset
* Next Steps


**Read Every Cell FULLY before executing it**

For more informations about APIs, please check the [documentation](https://docs.aws.amazon.com/forecast/latest/dg/what-is-forecast.html)






## Setup

Import the standard Python libraries that are used in this lesson.



In [1]:
import sys
import os
import json
import time
import pandas as pd
import glob

import boto3

# importing forecast notebook utility from notebooks/common directory
sys.path.insert( 0, os.path.abspath("../../common") )
import util

Configure the S3 bucket name and region name for this lesson.

- If you don't have an S3 bucket, create it first on S3. If you used CloudFormation Wizard to set up the environment, use same bucket name as you specified in the setup process.
- Although we have set the region to us-west-2 as a default value below, you can choose any of the regions that the service is available in.

In [2]:
##Code needs to be fixed
text_widget_bucket = util.create_text_widget( "bucket_name", "input your S3 bucket name" )
text_widget_region = util.create_text_widget( "region", "input region name.", default_value="us-west-2" )

Text(value='', description='bucket_name', placeholder='input your S3 bucket name')

Text(value='us-west-2', description='region', placeholder='input region name.')

In [ ]:
##Code needs to be fixed
bucket_name = text_widget_bucket.value
#assert bucket_name, "bucket_name not set."

region = text_widget_region.value
#assert region, "region not set."


The last part of the setup process is to validate that your account can communicate with Amazon Forecast, the cell below does just that.

In [ ]:
##Code needs to be fixed
session = boto3.Session(region_name=region) 
forecast = session.client(service_name='forecast') 
forecastquery = session.client(service_name='forecastquery')

## Data Preparation
First we combine all the p&gsales_*.csv files using Pandas. This is done inorder to get one data frame which is chronologically sorted as a time series from 2016 to 2020

In [12]:
#df = pd.read_excel("./p&gsales_1.xlsx", index_col = "Customer")
df1 = pd.read_csv("./p&gsales_1.csv", index_col="Date")
df2 = pd.read_csv("./p&gsales_2.csv", index_col="Date")
df3 = pd.read_csv("./p&gsales_3.csv", index_col="Date")
df4 = pd.read_csv("./p&gsales_4.csv", index_col="Date")

frames = [df1, df2, df3, df4]
df = pd.concat(frames)

Check the first 5 entries i.e. make sure they start as p&gsales_1.csv starts

In [25]:
df.head()

,Customer,Name,Material,Article,Billed Value,Billed Qty
Date,,,,,,
03/01/2016,60000707,LULU HYPERMARKET-SOHAR,179045,TIDE NS FLA 1.5KG,40.752,24.0
03/01/2016,60000707,LULU HYPERMARKET-SOHAR,179047,TIDE NS FLA WED 3KG,13.296,4.0
03/01/2016,60000707,LULU HYPERMARKET-SOHAR,179051,TIDE NS MB 260GM,18.176,64.0
03/01/2016,60000707,LULU HYPERMARKET-SOHAR,179053,TIDE NS MB 3KG,13.296,4.0
03/01/2016,60000707,LULU HYPERMARKET-SOHAR,179058,TIDE NS JASMINE 1.5KG,40.752,24.0


Check the last 5 entries i.e. make sure they end as p&gsales_4.csv ends

In [27]:
df.tail()

,Customer,Name,Material,Article,Billed Value,Billed Qty
Date,,,,,,
30/04/2020,60097965,ROYAL STAR INT'L LLC - NESTO - SAAD,212450,PANTENE ATLAS ANTI DANDRUFF SHMP 400ML,-8.556,-6.0
30/04/2020,60097965,ROYAL STAR INT'L LLC - NESTO - SAAD,248420,HE SHMP ARABICA COFFEE FRUIT 400ML,-22.128,-12.0
30/04/2020,60097965,ROYAL STAR INT'L LLC - NESTO - SAAD,266730,ADIDAS AP ROLL-ON FEMALE CLIMACOOL 50ML,-6.240,-6.0
30/04/2020,60097965,ROYAL STAR INT'L LLC - NESTO - SAAD,267363,OB KIDS FROZEN & CARS 75ML,-15.096,-12.0
30/04/2020,60097965,ROYAL STAR INT'L LLC - NESTO - SAAD,283299,GILL BLUE3 COMFORT DISPOSABLE 6S,-27.360,-12.0


Print out the dimensions of this dataframe

In [28]:
print(df.shape)


(3056772, 6)


Further Cleaning the data and formatting columns to correct data types